Python: 3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]

|모듈|버젼|
|----|----|
|pandas|0.25.1|
|numpy|1.18.5|
|sklearn|0.21.3|
|scipy|1.5.2|
|mlxtend|0.15.0.0|
|statsmodels|0.11.1|
|imblearn|0.5.0|
|xgboost|0.80|

In [1]:
import pandas as pd
import numpy as np
import sklearn
import scipy 
import mlxtend
import statsmodels
import imblearn
import xgboost as xgb
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, imblearn, xgb]:
    print(i.__name__,  i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
imblearn 0.5.0
xgboost 0.80


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

# 대문항 세트2

사람의 신체적 행위는 목적과 의도에서 비롯됩니다. 

예를 들어, 철수가 주머니에서 손을 꺼내 앞으로 내민 상황에서, 

영희는 철수의 행위를 통해 악수하고자 하는 의도를 파악하고 본인도 손을 내밉니다. 

즉, 사람의 신체적 행위를 관찰하면 그 의도를 유추할 수 있습니다. 

본 문제에서는 손의 움직임을 3차원 좌표로 저장할 수 있는 마커를 실험자 손에 부착하고, 

측정된 값을 분석하여 손의 움직임에 따른 의도를 예측하고자 합니다. 

예측하고자 하는 의도(label)의 종류는 총 6가지로 ‘grasp (잡기), touch (터치하기), openclose (여닫기), shake (흔들기), release (놓기), none (의도 없음)’ 중 하나입니다.

피험자의 손에 총 3개의 마커 ‘A, B, C’를 부착한 후, 각 의도마다 40번의 trial을 진행하여 3차원 좌표 데이터(x, y, z)를 수집하였습니다. 

또한 각 trial에서 마커당 0.1초 간격으로 총 10회에 걸쳐 좌표 데이터를 측정하였습니다. 

즉, 각 trial로부터 총 90개의 피처(3개 마커 * 10개 프레임 * 3개 좌표) 값을 수집하였습니다. 

관측 카메라에 특정 마커가 보이지 않을 경우 해당 프레임에서의 x, y, z 좌표는 모두 1000으로 대체하였습니다. 

위 방식에 의해 수집된 행위 측정 데이터셋은 아래와 같이 각 trial(행) 마다 수집된 마커 측정 좌표(90개), 피험자의 무게중심 좌표(3개) 및 의도(1개)로 구성되어 있습니다.

**prof2105_2.csv** (구분자: Comma(“,”), 240 Rows, 94 Columns, UTF-8 인코딩)

- 컬럼명 규칙(첫 4개 컬럼 제외): joint_[마커]_[프레임]_[좌표]

  (마커는 A, B C 중 하나, 프레임은 t0, t1, …, t9 중 하나, 좌표는 x, y, z 중 하나)

|컬럼명|설명|Type|
|------|----|----|
|label|의도(grasp/touch/openclose/shake/release/none)|string|
|object_x|피험자 무게중심의 x좌표 (meter)|double|
|object_y|피험자 무게중심의 y좌표 (meter)|double|
|object_z|피험자 무게중심의 z좌표 (meter)|double|
|joint_A_t0_x|A 마커 t0 프레임의 x좌표 (meter)|double|
|joint_A_t0_y|A 마커 t0 프레임의 y좌표 (meter)|double|
|joint_A_t0_z|A 마커 t0 프레임의 z좌표 (meter)|double|
|joint_B_t0_x|B 마커 t0 프레임의 x좌표 (meter)|double|
|…|…|…|
|joint_C_t9_y|C 마커 t9 프레임의 y좌표 (meter)|double|
|joint_C_t9_z|C 마커 t9 프레임의 z좌표 (meter)|double|

**prof2105_2_test.csv** (구분자: Comma(“,”), 60 Rows, 93 Columns, UTF-8 인코딩)

- prof2105_2.csv 데이터셋과 변수 구성은 동일하나, "의도"(label)에 관한 정보는 포함하지 않은 데이터셋으로 구성되었으며, 

  8번(Kaggle형) 문항에서 사용


In [3]:
df_motion_org = pd.read_csv('data/prof2105_2.csv')
df_motion_org.head()

,label,object_x,object_y,object_z,joint_A_t0_x,joint_A_t0_y,joint_A_t0_z,joint_B_t0_x,joint_B_t0_y,joint_B_t0_z,...,joint_C_t8_z,joint_A_t9_x,joint_A_t9_y,joint_A_t9_z,joint_B_t9_x,joint_B_t9_y,joint_B_t9_z,joint_C_t9_x,joint_C_t9_y,joint_C_t9_z
0,grasp,0.12873,0.11474,0.02848,-0.13834,-0.11134,0.08939,-0.10300,-0.06286,0.10328,...,0.10519,-0.13938,-0.11033,0.08883,-0.10346,-0.06027,0.09901,-0.16852,-0.04275,0.10519
1,grasp,0.12873,0.11474,0.02848,-0.12940,-0.11468,0.09201,-0.09848,-0.06208,0.10490,...,0.10813,-0.13343,-0.10742,0.08939,-0.09993,-0.05745,0.10420,-0.16411,-0.04204,0.10867
2,grasp,0.12873,0.11474,0.02848,-0.13327,-0.11975,0.09481,-0.09872,-0.07082,0.11248,...,0.11419,-0.13287,-0.10913,0.09303,-0.09923,-0.05964,0.10638,-0.16402,-0.04483,0.11410
3,grasp,0.12873,0.11474,0.02848,-0.13066,-0.11123,0.08942,-0.09827,-0.06138,0.10649,...,0.11308,-0.12989,-0.10978,0.08796,-0.09702,-0.05865,0.10346,-0.16512,-0.04707,0.11308
4,grasp,0.12873,0.11474,0.02848,-0.13736,-0.10460,0.11447,-0.09483,-0.05778,0.13083,...,0.15276,-0.13738,-0.10311,0.11303,-0.09506,-0.05556,0.12750,-0.15968,-0.03544,0.15276


## 공통 전처리

**(prof2105_2.csv 를 이용하여)** 다음의 전처리를 순서대로 수행하시오.

**Step B-1** 

정확한 분석 결과를 위해, 관측 카메라에 보이지 않아 1000으로 입력된 데이터를 처리합니다. 

prof2105_2.csv 데이터셋의 모든 컬럼에 대해 1000으로 수집된 모든 데이터를 결측치(null/NaN)로 변환하여 df_a_pre 데이터셋으로 저장하시오.

(240 rows, 94 columns)

Note. df_a_pre의 결측치는 총 519개 입니다.


In [4]:
# 1000을 na 값으로 치환합니다.
df_a_pre = df_motion_org.replace({1000: np.nan})
df_a_pre.isna().sum().sum(), df_a_pre.shape

(519, (240, 94))

**Step B-2**

마커에서 수집된 좌표 데이터들은 실험이 이뤄진 시간 및 공간에서의 절대좌표입니다. 

이번 단계에서는 피험자의 위치를 기준으로 하는 상대좌표 컬럼들을 생성합니다. 

즉, 전단계에서 생성한 df_a_pre 데이터셋을 사용하여 상대좌표 컬럼(컬럼명: dist_\[마커\]_\[프레임\]_\[좌표\])들을 생성하시오. 

마커(A, B, C)별 프레임(t0, …, t9)의 모든 좌표(x, y, z)에 대해 생성하며, 

아래는 마커 A, 프레임 t0, 좌표 x, y, z에 대한 예시입니다. 

(단, 결측치(null/NaN)로 인해 계산이 불가한 경우는 컬럼값을 결측치로 유지합니다.)

```
dist_A_t0_x = joint_A_t0_x - object_x
dist_A_t0_y = joint_A_t0_y - object_y
dist_A_t0_z = joint_A_t0_z - object_z
```

Note. 변수명의 마지막 알파벳이 'x'이면 object_x를, 'y'이면 object_y를, 'z'이면 object_z를 기존 좌표에서 빼줍니다.

df_a_pre 데이터셋의 label과 위 수식에 따라 생성한 컬럼 90개(3개 마커 * 10개 프레임 * 3개 좌표)를 추출하여 

df_b_pre 데이터셋으로 저장하시오. (240 rows, 91 columns)

 Note. dist_A_t0_x의 첫번째 데이터를 소수점 둘째자리까지 반올림한 값은 -0.27 입니다.


In [5]:
# subtract(column 단위로 뺄샘 vectorize 연산을 제공기능을 활용합니다.)
# add, mul, div : 덧샘, 곱샘, 나눗셈
# joint_로 시작하고 _x, _y, _z로 끝나는 행들을 반복하여 불러오고 subtract연산을 이용해 object_*로 빼주어 
# 문제에서 제시한 조건으로 좌표를 중앙 정렬합니다.
df_b_pre = pd.concat(
    [df_a_pre['label']] +
    [
        df_a_pre[[i for i in df_a_pre.columns if i.startswith('joint_') & i.endswith(j)]]\
                .subtract(df_a_pre['object{}'.format(j)], axis=0).rename(columns=lambda x: 'dist' + x[5:])
        for j in ['_x', '_y', '_z']
    ], axis=1
)
df_b_pre

,label,dist_A_t0_x,dist_B_t0_x,dist_C_t0_x,dist_A_t1_x,dist_B_t1_x,dist_C_t1_x,dist_A_t2_x,dist_B_t2_x,dist_C_t2_x,...,dist_C_t6_z,dist_A_t7_z,dist_B_t7_z,dist_C_t7_z,dist_A_t8_z,dist_B_t8_z,dist_C_t8_z,dist_A_t9_z,dist_B_t9_z,dist_C_t9_z
0,grasp,-0.267070,-0.231730,-0.297970,-0.267400,-0.231480,-0.297590,-0.266700,-0.231720,-0.297240,...,0.076710,0.060350,0.070530,0.076710,0.059990,0.070150,0.076710,0.060350,0.070530,0.076710
1,grasp,-0.258130,-0.227210,-0.294080,-0.259790,-0.227250,-0.295670,-0.259710,-0.227250,-0.293860,...,0.080570,0.060550,0.075720,0.080230,0.060550,0.075720,0.079650,0.060910,0.075720,0.080190
2,grasp,-0.262000,-0.227450,-0.291720,-0.260960,-0.227020,-0.292260,-0.260630,-0.227040,-0.292240,...,0.085340,0.063280,0.077420,0.084880,0.063640,0.077420,0.085710,0.064550,0.077900,0.085620
3,grasp,-0.259390,-0.227000,-0.294530,-0.259010,-0.226750,-0.293810,-0.258620,-0.226250,-0.293850,...,0.083770,0.059480,0.074980,0.084600,0.059480,0.074130,0.084600,0.059480,0.074980,0.084600
4,grasp,-0.266090,-0.223560,-0.288690,-0.266430,-0.223320,-0.289350,-0.267110,-0.224010,-0.288690,...,0.124630,0.084550,0.098660,0.124280,0.084550,0.099020,0.124280,0.084550,0.099020,0.124280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,touch,-0.212388,-0.242726,-0.202922,-0.216989,-0.245489,-0.203501,-0.224720,-0.251043,-0.210535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,touch,-0.209607,-0.240514,-0.200602,-0.205660,-0.237889,-0.197708,-0.215629,-0.248209,-0.200784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,touch,-0.207525,-0.238294,-0.200248,-0.207525,-0.237717,-0.199630,-0.206972,-0.237139,-0.199003,...,0.125172,0.120184,0.130692,0.125716,0.119897,0.130694,0.125718,0.119322,0.130395,0.125399
238,touch,-0.207525,-0.238872,-0.200248,-0.207525,-0.238294,-0.199630,-0.206972,-0.237717,-0.199003,...,0.125172,0.119208,0.130689,0.125172,0.119897,0.130692,0.125718,0.119609,0.130395,0.125718


**Step B-3**

StepB-1와 StepB-2에서 생성한 df_a_pre, df_b_pre 데이터셋의 모든 결측치(null/NaN)들을 0으로 대체하고 

각각 df_a, df_b 데이터셋으로 저장하시오.

위의 전처리 과정을 완료한 데이터 셋 df_a, df_b와 prof2105_2_test.csv를 이용하여 문제를 풀이하시오. 

(이후 문항에서의 지칭을 위하여 데이터 셋 이름을 정의한 것으로, 제출 소스코드에서 반드시 위의 이름을 따를 필요는 없다)


In [6]:
df_a = df_a_pre.fillna(0)
df_b = df_b_pre.fillna(0)

## 문제 5번 15점

**(df_a 데이터셋을 이용하여)** 좌표 x, y, z 중 마커별로 큰 차이가 없는 좌표가 있다면 

이후 분석에 있어 feature 수를 줄이는 데 참고할 수 있습니다. 

이를 위해 마커 간 x좌표들의 관계를 분석하여 feature 수를 줄이는 데 참고하고자 합니다. 

이를 위해 df_a를 사용하여, A마커의 x좌표로 B 마커의 x좌표를 예측하는 선형 회귀(Linear regression)모델을 생성하겠습니다. 

아래의 단계들을 순서대로 수행한 후 질문에 답하시오.

**Step 5-1**

아래 조건에 따라 train data와 test data를 구성합니다.

- train data(1,680 rows, 2 columns)는 아래 두 개의 컬럼으로 구성

   joint_A_x: A 마커 0\~6번 프레임(t0\~t6)의 x좌표들을 순서대로 행 결합한 컬럼
   
   joint_B_x: B 마커 0\~6번 프레임(t0\~t6)의 x좌표들을 순서대로 행 결합한 컬럼
   
 
- test data(720 rows, 2 columns)는 아래 두 개의 컬럼으로 구성

   joint_A_x: A 마커 7\~9번 프레임(t7\~t9)의 x좌표들을 순서대로 행 결합한 컬럼
   
   joint_B_x: B 마커 7\~9번 프레임(t7\~t9)의 x좌표들을 순서대로 행 결합한 컬럼


In [7]:
# melt를 이용해서 7개의 컬럼을 1의 컬럼으로 형태변환 하여 joint_A_x와 joint_B_x를 만들어 냅니다.

df_train_5 = pd.concat([
    df_a[['joint_A_t{}_x'.format(i) for i in range(7)]].melt()['value'].rename('joint_A_x'),
    df_a[['joint_B_t{}_x'.format(i) for i in range(7)]].melt()['value'].rename('joint_B_x')
], axis=1)

df_test_5 = pd.concat([
    df_a[['joint_A_t{}_x'.format(i) for i in range(7, 10)]].melt()['value'].rename('joint_A_x'),
    df_a[['joint_B_t{}_x'.format(i) for i in range(7, 10)]].melt()['value'].rename('joint_B_x')
], axis=1)
df_train_5.shape, df_test_5.shape

((1680, 2), (720, 2))

In [8]:
# 이용해서 7개의 컬럼을 Series로 뽑아와서 각각을 결합시켜, joint_A_x와 joint_B_x를 만들어 냅니다.

df_train_5 = pd.concat([
    pd.concat([df_a['joint_A_t{}_x'.format(i)] for i in range(7)]).rename('joint_A_x'),
    pd.concat([df_a['joint_B_t{}_x'.format(i)] for i in range(7)]).rename('joint_B_x')
], axis=1)

df_test_5 = pd.concat([
    pd.concat([df_a['joint_A_t{}_x'.format(i)] for i in range(7, 10)]).rename('joint_A_x'),
    pd.concat([df_a['joint_B_t{}_x'.format(i)] for i in range(7, 10)]).rename('joint_B_x')
], axis=1)
df_train_5.shape, df_test_5.shape

((1680, 2), (720, 2))

**Step 5-2**

위 Step 5-1에서 구성한 train data를 사용하여, 

독립변수 joint_A_x로 종속변수 joint_B_x를 예측하는 선형회귀(Linear Regression) 모델을 학습합니다. 

**Python 함수가이드**
```
from sklearn.linear_model import LinearRegression 
문제 지시 사항 외 Default 파라미터 사용
```

In [9]:
from sklearn.linear_model import LinearRegression 

# 선형회귀모델을 학습합니다.
X_lr = ['joint_A_x']
reg_lr = LinearRegression()
reg_lr.fit(df_train_5[X_lr], df_train_5['joint_B_x'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**Step 5-3**

위 Step 5-2에서 학습한 모델을 Step 5-1의 test data에 적용하여 'joint_B_x'를 예측한 후, 

test data에 대한 결정계수(Coefficient of determination, R^2  score)와 MAE(Mean Absolute Error)를 구합니다. 

(아래 수식 참고, MAE에서 f : 예측값, y: 실제값)

$R^2  score=\frac{SSR}{SST}=1-\frac{SSE}{SST}$

$MAE=\frac{1}{N}\sum_{i=1}^N|f_i-y_i|$

위 Step 5-3을 통해 얻은 결정계수와 MAE의 합을 계산하여 소수점 넷째 자리에서 반올림하여 셋째 자리까지 기술하시오.

(답안예시) 1.234


In [10]:
from sklearn.metrics import r2_score, mean_absolute_error

# 예측합니다.
y_hat = reg_lr.predict(df_test_5[X_lr])
# r2를 측정합니다.
r2 = r2_score(df_test_5['joint_B_x'], y_hat)
# mae를 측정합니다.
mae = mean_absolute_error(df_test_5['joint_B_x'], y_hat)
r2, mae, r2 + mae

(0.8798276323823012, 0.028634558914864734, 0.908462191297166)

**0.908**

## 문제 6번 10점

(df_a, df_b 데이터셋을 이용하여) 이번 문제에서는 각 마커의 ‘절대좌표(df_a), 상대좌표(df_b)’를 활용하여, 

각 정보에 대한 피험자의 의도를 예측하는 ‘다층 퍼셉트론(Multi-Layer Perceptron, MLP)’ 분류모델을 생성합니다. 

MLP 모델은 인간의 뉴런(neuron)들이 행동하는 것처럼 여러 개의 은닉층(Hidden Layer)들을 사용한 알고리즘으로, 주어진 데이터로부터 각 은닉층에 대한 가중치를 계산해 재적용하는 방식으로 모델을 학습합니다. df_a, df_b 각각의 데이터셋에 대해 아래의 단계들을 순서대로 수행한 후 질문에 답하시오.

**Step 6-1**

df_a와 df_b 각각을 데이터셋을 7:3 비율에 맞춰 df_a_train, df_a_test와 df_b_train, df_b_test로 분할합니다.

**Python 함수가이드**
```
from sklearn.model_selection import train_test_split 사용
- test_size=0.3
- random_state=42
문제 지시 사항 외 Default 파라미터 사용
```

In [11]:
from sklearn.model_selection import train_test_split

df_a_train, df_a_test = train_test_split(df_a, test_size=0.3, random_state=42)
df_b_train, df_b_test = train_test_split(df_b, test_size=0.3, random_state=42)

**Step 6-2**

위 Step 6-1에서 얻은 train data로 의도(label)를 예측하는 ‘다층 퍼셉트론(Multi-Layer Perceptron, MLP)’ 분류 모델을 학습합니다. 

df_a_train은 독립변수로는 ‘label, object_x, object_y, object_z’를 제외한 모든 feature들을 입력변수로, label을 대상변수로 

하여 MLP 모델 model_a를 학습합니다.

df_b_train은 독립변수로는 'label'을 제외한 모든 feature들을 입력변수로, label은 대상변수로 하여 MLP 모델 model_b를 학습합니다.


(입력변수의 수: df_a: 90개, df_b: 90개) 

**Python 함수가이드**
```
from sklearn.neural_network import MLPClassifier
- Feature Column 입력 순서 : 사전 오름차순으로
- activation='tanh'
- random_state=1234
문제 지시 사항 외 Default 파라미터 사용
```

In [12]:
from sklearn.neural_network import MLPClassifier
X_mlp_a = [i for i in df_a_train.columns if i not in ['label', 'object_x', 'object_y', 'object_z']]
X_mlp_b = [i for i in df_b_train.columns if i not in ['label']]
X_mlp_a.sort()
X_mlp_b.sort()
reg_mlp_a = MLPClassifier(
    activation='tanh', random_state=1234
)
reg_mlp_a.fit(df_a_train[X_mlp_a], df_a_train['label'])

reg_mlp_b = MLPClassifier(
    activation='tanh', random_state=1234
)
reg_mlp_b.fit(df_b_train[X_mlp_b], df_b_train['label'])

c:\users\student\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\student\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1234, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)


**Step 6-3**

위 Step 6-2에서 학습한 model_a로 df_a_test 에서 '의도별 재현율(recall)의 평균'을 구하고,

model_b로 df_b_test를 에서 '의도별 재현율(recall)의 평균'을 구합니다.



In [16]:
from sklearn.metrics import recall_score
recalls_a_ = recall_score(df_a_train['label'], reg_mlp_a.predict(df_a_train[X_mlp_a]), average=None)
recalls_b_ = recall_score(df_b_train['label'], reg_mlp_b.predict(df_b_train[X_mlp_b]), average=None)
np.mean(recalls_a_), np.mean(recalls_b_)

(0.9575243147656941, 0.9701395730706075)

In [19]:
recalls_a_ = recall_score(df_a_test['label'], reg_mlp_a.predict(df_a_test[X_mlp_a]), average=None)
recalls_b_ = recall_score(df_b_test['label'], reg_mlp_b.predict(df_b_test[X_mlp_b]), average=None)
recall_a = np.mean(recalls_a_)
recall_b = np.mean(recalls_b_)
recall_a, recall_b

(0.9618326118326119, 0.9552188552188552)

위 Step 6-3을 통해 얻은 각 데이터셋의 test data에 대한 '의도별 재현율(recall)의 평균'(총 2개) 중 

가장 큰 값을 소수점 넷째 자리에서 반올림하여 셋째 자리까지 기술하시오.

 (답안예시) 0.423

In [21]:
recall_a, recall_b, max(recall_a, recall_b)

(0.9618326118326119, 0.9552188552188552, 0.9618326118326119)

**0.962**

# 쉬는 시간: ~ 16:42

## 문제 7번 5점

(df_a, df_b 데이터셋을 이용하여) 이번에는 각 마커의 '절대좌표(df_a), 상대좌표(df_b)'를 모두 활용한 MLP 분류모델을 학습하여 

각각의 모델을 생성했을 때에 비해 성능향상이 있는지 확인해 보겠습니다. 

아래의 단계들을 순서대로 수행한 후 질문에 답하시오.

**Step 7-1**

2개의 데이터셋(df_a, df_b)을 순서대로 연결합 합니다. 

이때 중복되는 의도(label) 컬럼은 하나만 남기고, 

df_a의 'object_x, object_y, object_z' 컬럼은 제외한 후 결합합니다. 

(240 rows, 181 columns)


**Step 7-2**

위 Step 7-1에서 결합한 데이터셋을 7:3 비율에 맞춰 train data와 test data로 분할합니다.

**Python 함수가이드**
```
from sklearn.model_selection import train_test_split 사용
- test_size=0.3
- random_state=42
문제 지시 사항 외 Default 파라미터 사용
```

**Step 7-3**

위 Step 7-2에서 얻은 train data로 의도(label)를 예측하는 '다층 퍼셉트론(Multi-Layer Perceptron, MLP)' 분류 모델을 학습합니다. 

이 때, 독립변수로는 의도(label)를 제외한 모든 feature(180개)들을 사용합니다.

**Python 함수가이드**
```
from sklearn.neural_network import MLPClassifier
- Feature Column 입력 순서 : df_a, df_b 순서대로, 
각각 사전 오름차순으로 입력 
- activation='tanh'
- random_state=1234
문제 지시 사항 외 Default 파라미터 사용
```


**Step 7-4**

위 Step 7-3에서 학습한 모델을 Step 7-2의 test data에 적용하여 의도(label)를 예측한 후, 실제 의도와 비교하여 

'의도별 재현율(recall)의 평균'을 구합니다.

위 Step 7-4를 통해 얻은 test data에 대한 '의도별 재현율의 평균'을 소수점 넷째 자리에서 반올림하여 셋째 자리까지 기술하시오.


(답안예시) 0.789